# 102 Spark optimizations

The goal of this lab is to understand some of the optimization mechanisms of Spark.

- Scala
    - [Spark programming guide](https://spark.apache.org/docs/latest/rdd-programming-guide.html)
    - [RDD APIs](https://spark.apache.org/docs/latest/api/scala/org/apache/spark/rdd/RDD.html)
    - [PairRDD APIs](https://spark.apache.org/docs/latest/api/scala/org/apache/spark/rdd/PairRDDFunctions.html)
- Python
    - [Spark programming guide](https://spark.apache.org/docs/3.5.0/rdd-programming-guide.html)
    - [All RDD APIs](https://spark.apache.org/docs/3.5.0/api/python/reference/api/pyspark.RDD.html)

Use `Tab` for autocompletion, `Shift+Tab` for documentation.

In [1]:
from pyspark.sql import SparkSession

# .master("local[N]") <-- ask for N cores on the driver
spark = SparkSession.builder \
.master("local[4]") \
.appName("Local Spark") \
.config('spark.ui.port', '4040') \
  .getOrCreate()
sc = spark.sparkContext

sc

<SparkContext master=local[4] appName=Local Spark>

## The weather dataset

Download the following ZIP files and unzip them inside the "datasets/big" folder of this repo (which is not committed).
- [weather-sample1.txt](https://big.csr.unibo.it/downloads/bigdata/weather-datasets-s1.zip) <-- start from this!
- [weather-sample10.txt](https://big.csr.unibo.it/downloads/bigdata/weather-datasets-s10.zip)
- [weather-full.txt](https://big.csr.unibo.it/downloads/bigdata/weather-datasets-full.zip)
  
The weather datasets are textual files with weather data from all over the world in year 2000 (collected from the [National Climatic Data Center](ftp://ftp.ncdc.noaa.gov/pub/data/noaa/) of the USA. The full one weighs 13GB, the other are samples of 10% (1.3GB) and 1% (130MB) respectively.
  - Sample row: 005733213099999**19580101**03004+51317+028783FM-12+017199999V0203201N00721004501CN0100001N9 **-0021**1-01391102681
  - The date in YYYYMMDD format is located at 0-based position 15-23
  - The temperatue in x10 Celsius degrees is located at 0-based positions 87-92

In the dataset folder you also have *weather-stations.csv*; it is a structured file with the description of weather stations collecting the weather data.

In [2]:
# WEATHER structure: (usaf,wban,year,month,day,airTemperature,airTemperatureQuality)
def parseWeather(row):
    usaf = row[4:10]
    wban = row[10:15]
    year = row[15:19]
    month = row[19:21]
    day = row[21:23]
    airTemperature = row[87:92]
    airTemperatureQuality = row[92]

    return (usaf,wban,year,month,day,int(airTemperature)/10,airTemperatureQuality == '1')

# STATION structure: (usaf,wban,city,country,state,latitude,longitude,elevation,date_begin,date_end) 
def parseStation(row):
    def getDouble(str):
        return 0 if len(str)==0 else float(str)
    
    columns = [ x.replace("\"","") for x in row.split(",") ]
    latitude = getDouble(columns[6])
    longitude = getDouble(columns[7])
    elevation = getDouble(columns[8])
    return (columns[0],columns[1],columns[2],columns[3],columns[4],latitude,longitude,elevation,columns[9],columns[10])  

In [3]:
rddWeather = sc.\
  textFile("../../../../datasets/big/weather-sample1.txt").\
  map(lambda x: parseWeather(x))
rddStation = sc.\
  textFile("../../../../datasets/weather-stations.csv").\
  map(lambda x: parseStation(x))

## 102-1 Simple job optimization

Optimize the two jobs (avg temperature and max temperature) by avoiding the repetition of the same computations and by enforcing a partitioning criteria.
- There are multiple methods to repartition an RDD: check the ```coalesce```, ```partitionBy```, and ```repartition``` methods on the documentation and choose the best one.
- Verify your persisted data in the web UI
- Verify the execution plan of your RDDs with ```rdd.toDebugString``` (shell only) or on the web UI

In [4]:
# Average temperature for every month
rddCached=rddWeather.\
  filter(lambda x: x[5]<999).\
  map(lambda x: (x[3], (x[5],1))).\
  partitionBy(8).\
  persist()
#Use partitionBy 8 because 4x2
rddCached.reduceByKey(lambda v1, v2: (v1[0]+v2[0], v1[1]+v2[1])).\
  mapValues(lambda v: round(v[0]/v[1],2)).\
  collect()

[('10', 8.58),
 ('02', 0.41),
 ('06', 11.95),
 ('07', 14.1),
 ('08', 13.78),
 ('11', 4.38),
 ('09', 10.62),
 ('01', 0.31),
 ('05', 9.64),
 ('12', 1.74),
 ('04', 4.91),
 ('03', 1.89)]

In [5]:
# Maximum temperature for every month
rddWeather.\
  filter(lambda x: x[5]<999).\
  map(lambda x: (x[3], (x[5],1)))
rddCached.reduceByKey(lambda x, y: y if x<y else x).\
  collect()

[('10', (20.0, 1)),
 ('02', (13.2, 1)),
 ('06', (31.4, 1)),
 ('07', (29.2, 1)),
 ('08', (23.0, 1)),
 ('11', (14.0, 1)),
 ('09', (30.0, 1)),
 ('01', (12.0, 1)),
 ('05', (34.2, 1)),
 ('12', (14.0, 1)),
 ('04', (23.0, 1)),
 ('03', (15.2, 1))]

## 102-2 RDD preparation

Check the five possibilities to prepare the Station RDD for subsequent processing and identify the best one.

In [7]:
num_partitions = 8

# [0] and [1] are the fields composing the key; [3] and [7] are country and elevation, respectively
rddS1 = rddStation.\
  keyBy(lambda x: x[0] + x[1]).\
  partitionBy(num_partitions).\
  cache().\
  map(lambda kv: (kv[0],(kv[1][3],kv[1][7])))
rddS2 = rddStation.\
  keyBy(lambda x: x[0] + x[1]).\
  map(lambda kv: (kv[0],(kv[1][3],kv[1][7]))).\
  cache().\
  partitionBy(num_partitions)
rddS3 = rddStation.\
  keyBy(lambda x: x[0] + x[1]).\
  partitionBy(num_partitions).\
  map(lambda kv: (kv[0],(kv[1][3],kv[1][7]))).\
  cache()
rddS4 = rddStation.\
  keyBy(lambda x: x[0] + x[1]).\
  map(lambda kv: (kv[0],(kv[1][3],kv[1][7]))).\
  partitionBy(num_partitions).\
  cache()
rddS5 = rddStation.\
  map(lambda x: (x[0] + x[1], (x[3],x[7]))).\
  partitionBy(num_partitions).\
  cache()

## 102-3 Joining RDDs

Define the join between rddWeather and rddStation and compute:
- The maximum temperature for every city
- The maximum temperature for every city in the UK: 
  - ```StationData.country == "UK"```
- Sort the results by descending temperature
  - ```map(lambda kv: (kv[1],kv[0]))``` to invert key with value and vice versa

Hints & considerations:
- Keep only temperature values <999
- Join syntax: ```rdd1.join(rdd2)```
  - Both RDDs should be structured as key-value RDDs with the same key: usaf + wban
- Consider partitioning and caching to optimize the join
  - [Scala only] Careful: it is not enough for the two RDDs to have the same number of partitions; they must have the same partitioner! To create a partitioning function, you must ```import org.apache.spark.HashPartitioner``` and then define ```p = new HashPartitioner(n)``` where ```n``` is the number of partitions to create.
- Verify the execution plan of the join in the web UI

In [23]:
rddStationKey=rddStation.map(lambda x: (x[0] + x[1], (x[3],x[7])))

rddJoined=rddWeather.keyBy(lambda x: x[0]+x[1]).join(rddStationKey)
#Needs to be the same key
rddJoined.collect()

[('02869099999',
  (('028690', '99999', '2000', '04', '01', 999.9, False), ('FI', 264.0))),
 ('02869099999',
  (('028690', '99999', '2000', '04', '01', 999.9, False), ('FI', 264.0))),
 ('02869099999',
  (('028690', '99999', '2000', '04', '01', -8.6, True), ('FI', 264.0))),
 ('02869099999',
  (('028690', '99999', '2000', '04', '01', 999.9, False), ('FI', 264.0))),
 ('02869099999',
  (('028690', '99999', '2000', '04', '01', 999.9, False), ('FI', 264.0))),
 ('02869099999',
  (('028690', '99999', '2000', '04', '02', -9.2, True), ('FI', 264.0))),
 ('02869099999',
  (('028690', '99999', '2000', '04', '02', 999.9, False), ('FI', 264.0))),
 ('02869099999',
  (('028690', '99999', '2000', '04', '02', 999.9, False), ('FI', 264.0))),
 ('02869099999',
  (('028690', '99999', '2000', '04', '02', -10.6, True), ('FI', 264.0))),
 ('02869099999',
  (('028690', '99999', '2000', '04', '02', 999.9, False), ('FI', 264.0))),
 ('02869099999',
  (('028690', '99999', '2000', '04', '02', -11.0, True), ('FI', 264.

## 102-4 Memory occupation

Use Spark's web UI to verify the space occupied by the provided RDDs.

*Warning*: in PySpark, StoraleLevels use serialization by default (see [documentation](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.StorageLevel.html)).

In [ ]:
from pyspark import StorageLevel

# Clear the cache
for (id, rdd) in sc._jsc.getPersistentRDDs().items():         
    rdd.unpersist()

memSerRdd = rddWeather.cache()
memRdd = memSerRdd.map(lambda x: x).persist(StorageLevel. MEMORY_AND_DISK_DESER)
diskRdd = memSerRdd.map(lambda x: x).persist(StorageLevel.DISK_ONLY)

## 102-5 Evaluating different join methods

Consider the following scenario:
- We have a disposable RDD of Weather data (i.e., it is used only once): ```rddW```
- And we have an RDD of Station data that is used many times: ```rddS```
- Both RDDs are cached (```collect()```is called to enforce caching)

We want to join the two RDDS. Which option is best?
- Simply join the two RDDs
- Enforce on ```rddW1``` the same partitioner of ```rddS``` (and then join)
- Exploit broadcast variables

In [ ]:
num_partitions = 8

rddW = rddWeather.\
  filter(lambda w: w[5]<999).\
  keyBy(lambda w: w[0]+w[1]).\
  cache()

rddS = rddStation.\
  keyBy(lambda s: s[0]+s[1]).\
  partitionBy(num_partitions).\
  cache()

# Collect to enforce caching
rddW.collect()
rddS.collect()

In [ ]:
# Is it better to simply join the two RDDs..
rddX = rddW.\
  join(rddS).\
  map(lambda kv: (kv[1][1][2],kv[1][0][5])).\
  reduceByKey(lambda x,y: min(x,y),1)
print(rddX.toDebugString().decode("unicode_escape"))

In [ ]:
rddX.collect()

In [ ]:
# ..to enforce on rddW1 the same partitioner of rddS..
rddX = rddW.\
  partitionBy(num_partitions).\
  join(rddS).\
  map(lambda kv: (kv[1][1][2],kv[1][0][5])).\
  reduceByKey(lambda x,y: min(x,y),1)
print(rddX.toDebugString().decode("unicode_escape"))

In [ ]:
rddX.collect()

In [ ]:
# ..or to exploit broadcast variables?
bRddS = sc.broadcast(rddS.map(lambda s: (s[0], s[1][2])).collectAsMap())
rddJ = rddW.\
  map(lambda kv: (bRddS.value.get(kv[0]), kv[1][5])).\
  filter(lambda x: x[0] is not None)

rddX = rddJ.\
  reduceByKey(lambda x,y: min(x,y),1)
print(rddX.toDebugString().decode("unicode_escape"))

In [ ]:
rddX.collect()